In [1]:
#file locations
filename = '../../../Applications/Assessment/loans.csv'
loan_profile = '../loan_report.html'

#Email settings
port = 2525 
smtp_server = "smtp.mailtrap.io"
login = "1a2b3c4d5e6f7g" # paste your login generated by Mailtrap
password = "1a2b3c4d5e6f7g" # paste your password generated by Mailtrap
subject = "ETL Pipeline Results"
sender_email = "sender@mailtrap.io"
receiver_email = "new@mailtrap.io"


#Database connections
dw_string = "host='localhost' dbname='dw1' user='dw' password='dwhouse'"

In [2]:
########## ETL pipeline
# ETL tools
import psycopg2
import pygrametl
from pygrametl.datasources import SQLSource, CSVSource, PandasSource
from pygrametl.tables import Dimension, FactTable, CachedDimension, BulkFactTable

# stats tools
import pandas as pd
import pandas_profiling

#email tools
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#OS tools
import os

# Ingest CSV input data file and modify data types
filename = os.path.abspath(filename)
names = ['End of Period','loannumber','region','countrycode','country','borrower','guarantor_code','guarantor','loantype','loanstatus','interestrate','Currency of Commitment','projectidsrc','projectname' ,'orig_principal_amt','cancelled_amt','undisbursed_amt','disbursed_amt','repaid_to_ibrd','due_to_ibrd','exchg_adjustment','borrower_oblig','sold_3rd_party','repaid_3rd_party','due_3rd_party','loans_held','First Repayment Date','Last Repayment Date','Agreement Signing Date','Board Approval Date','Effective Date','Closed Date','Last Disbursement Date']
data = pd.read_csv(filename, names=names, skiprows=1)
data['projectidsrc'] = data['projectidsrc'].astype(str).str.strip()
data['countrycode'] = data['countrycode'].astype(str).str.upper().str.strip()
data['borrower'] = data['borrower'].astype(str).str.upper().str.strip()
data['guarantor_code'] = data['guarantor_code'].astype(str).str.upper().str.strip()
data['guarantor'] = data['guarantor'].astype(str).str.upper().str.strip()
data['loantype'] = data['loantype'].astype(str).str.upper().str.strip()
data['loanstatus'] = data['loanstatus'].astype(str).str.upper().str.strip()
data['interestrate'] = data['interestrate'].astype(str).str.upper().str.strip()
# df['End of Period'] = pd.to_numeric(df['End of Period'])
# df['Due 3rd Party'] = pd.to_datetime(df['Due 3rd Party']) 

/home/itumusiime/Documents/python_projects/env/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/home/itumusiime/Documents/python_projects/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (12,13,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
############Configs
'''
port = 2525 
smtp_server = "smtp.mailtrap.io"
login = "1a2b3c4d5e6f7g" # paste your login generated by Mailtrap
password = "1a2b3c4d5e6f7g" # paste your password generated by Mailtrap

subject = "ETL Pipeline Results"
sender_email = "sender@mailtrap.io"
receiver_email = "new@mailtrap.io"
loan_profile = "../loan_report.html"
total = len(data.index)
'''
total = len(data.index)

In [4]:
##########Explore data --Commented out for brevity
'''
peek = data.head(20)
print(peek)
print(data.shape)
print(peek.shape)
types = data.dtypes
print(types)
types = data.dtypes
print(types)
pd.set_option( 'display.width' , 100)
pd.set_option( 'precision' , 3)
description = data.describe()
print(description)
class_counts = data.groupby( 'projectname' ).size()
print(class_counts)
data.isnull().sum()
print(data.head(3))
data.columns
'''

"\npeek = data.head(20)\nprint(peek)\nprint(data.shape)\nprint(peek.shape)\ntypes = data.dtypes\nprint(types)\ntypes = data.dtypes\nprint(types)\npd.set_option( 'display.width' , 100)\npd.set_option( 'precision' , 3)\ndescription = data.describe()\nprint(description)\nclass_counts = data.groupby( 'projectname' ).size()\nprint(class_counts)\ndata.isnull().sum()\nprint(data.head(3))\ndata.columns\n"

In [5]:
##########Stats
# profile the data - Statistics
loan_profile = os.path.abspath(loan_profile)
profile = pandas_profiling.ProfileReport(data,minimal=True)
profile.to_file(loan_profile)

In [6]:
##########Database
# datawarehouse connection
data.drop(columns=['Currency of Commitment','First Repayment Date','Last Repayment Date','Agreement Signing Date','Board Approval Date','Effective Date','Closed Date','Last Disbursement Date'])
dataPS = PandasSource(data)
dw_string = "host='localhost' dbname='dw1' user='dw' password='dwhouse'"
dw_pgconn = psycopg2.connect(dw_string)
dw_conn_wrapper = pygrametl.ConnectionWrapper(connection=dw_pgconn)


In [7]:
##########Utility Methods
# bulk insert method
def bulkloader(name, attributes, fieldsep, rowsep, nullval, filehandle):
    cursor = dw_conn_wrapper.cursor()
    cursor.copy_from(file=filehandle, table=name, sep=fieldsep,null="nullval",
                     columns=attributes)
    
# split the timestamp into its parts
def split_timestamp(row):
    timestamp = row['End of Period']
    timestamp = timestamp[:10]
    timestamp_split = timestamp.split('-')
    row['year'] = timestamp_split[0]
    row['month'] = timestamp_split[1]
    row['day'] = timestamp_split[2]

# for sending email

def send_mail(sender_email, receiver_email, subject,  filename, total):
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject

    # Add body to email
    body = f"hi there, Please find attached stats from the monthly ETL pipeline,{total} records processed"
    message.attach(MIMEText(body, 'plain'))

    filename = filename


    # We assume that the file is in the directory where you run your Python script from
    with open(filename, 'rb') as attachment:
        # The content type "application/octet-stream" means that a MIME attachment is a binary file
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())

    # Encode to base64
    encoders.encode_base64(part)

    # Add header 
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to your message and convert it to string
    message.attach(part)
    text = message.as_string()

    # send your email
    with smtplib.SMTP("smtp.mailtrap.io", 2525) as server:
        server.login(login, password)
        server.sendmail(
            sender_email, receiver_email, text
        )
    print('Sent') 

In [8]:
########## Dimension and Fact tables
# Dimension and Fact objects
loan_dimension = CachedDimension(
    name='loandim',
    key='loanid',
    attributes=['loannumber'],
    lookupatts=['loannumber'],
    prefill=True)

eop_dimension = CachedDimension(
    name='eopdim',
    key='eopid',
    attributes=['day', 'month', 'year'],
    lookupatts=['day','month','year'],
    prefill=True)

country_dimension = CachedDimension(
    name='countrydim',
    key='countryid',
    attributes=['countrycode', 'country'],
    lookupatts=['country'],
    prefill=True)

region_dimension = CachedDimension(
    name='regiondim',
    key='regionid',
    attributes=['region'],
    lookupatts=['region'],
    prefill=True)

project_dimension = CachedDimension(
    name='projectdim',
    key='projectid',
    attributes=['projectidsrc', 'projectname'],
    lookupatts=['projectidsrc'],
    prefill=True)

borrower_dimension = CachedDimension(
    name='borrowerdim',
    key='borrowerid',
    attributes=['borrower'],
    lookupatts=['borrower'],
    prefill=True)

guarantor_dimension = CachedDimension(
    name='guarantordim',
    key='guarantorid',
    attributes=['guarantorcode', 'guarantor'],
    lookupatts=['guarantor'],
    prefill=True)

loantype_dimension = CachedDimension(
    name='loantypedim',
    key='loantypeid',
    attributes=['loantype'],
    lookupatts=['loantype'],
    prefill=True)

loanstatus_dimension = CachedDimension(
    name='loanstatusdim',
    key='loanstatusid',
    attributes=['loanstatus'],
    lookupatts=['loanstatus'],
    prefill=True)

ir_dimension = CachedDimension(
    name='interestratedim',
    key='interestrateid',
    attributes=['interestrate'],
    lookupatts=['interestrate'],
    prefill=True)




fact_table = BulkFactTable(
    name='loanfact',
    keyrefs=['loanid', 'eopid', 'countryid','regionid','projectid','borrowerid','guarantorid','loantypeid','loanstatusid','interestrateid'],
    measures=['orig_principal_amt','cancelled_amt','undisbursed_amt','disbursed_amt','repaid_to_ibrd','due_to_ibrd','exchg_adjustment','borrower_oblig','sold_3rd_party','repaid_3rd_party','due_3rd_party','loans_held'],
    bulkloader=bulkloader,
    bulksize=1000000)
  

'''# for initial bulk load to improve performance
fact_table = FactTable(
        name='loanfact',
        keyrefs=['loanid', 'eopid', 'countryid','regionid','projectid','borrowerid','guarantorid','loantypeid','loanstatusid','interestrateid'],
        measures=['orig_principal_amt','cancelled_amt','undisbursed_amt','disbursed_amt','repaid_to_ibrd','due_to_ibrd','exchg_adjustment','borrower_oblig','sold_3rd_party','repaid_3rd_party','due_3rd_party','loans_held'])
'''  

"# for initial bulk load to improve performance\nfact_table = FactTable(\n        name='loanfact',\n        keyrefs=['loanid', 'eopid', 'countryid','regionid','projectid','borrowerid','guarantorid','loantypeid','loanstatusid','interestrateid'],\n        measures=['orig_principal_amt','cancelled_amt','undisbursed_amt','disbursed_amt','repaid_to_ibrd','due_to_ibrd','exchg_adjustment','borrower_oblig','sold_3rd_party','repaid_3rd_party','due_3rd_party','loans_held'])\n"

In [9]:
########## Load Data
for row in dataPS:
    split_timestamp(row)
    row['loanid'] = loan_dimension.ensure(row)
    row['eopid'] = eop_dimension.ensure(row)
    row['countryid'] = country_dimension.ensure(row, namemapping={'countrycode':'countrycode'})
    row['regionid'] = region_dimension.ensure(row)
    row['projectid'] = project_dimension.ensure(row)
    row['borrowerid'] = borrower_dimension.ensure(row)
    row['guarantorid'] = guarantor_dimension.ensure(row, namemapping={'guarantorcode':'guarantor_code'})
    row['loantypeid'] = loantype_dimension.ensure(row)
    row['loanstatusid'] = loanstatus_dimension.ensure(row)
    row['interestrateid'] = ir_dimension.ensure(row)
    #insert fact
    
    #use for fact Updates
    #fact_table.ensure(row)
    
    #use for bulkload
    fact_table.insert(row)
    
dw_conn_wrapper.commit()
dw_conn_wrapper.close()

ProgrammingError: column "borrowerid" of relation "loanfact" does not exist


In [ ]:
##########Send email
# send_mail(sender_email, receiver_email, subject,  loan_profile, total)